In [25]:
from preprocessing import excel_to_df
import pandas as pd
import numpy as np

In [42]:
shares_out = 615
g_perp = 0.025
beta = 1.34
market_cap = 233645

In [3]:
nvdaIS, nvdaBS, nvdaCF = excel_to_df('tickers/NVDA.xlsx')

C:\Users\Andi\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
CF_OA = nvdaCF['Cash from Operating Activities']
CAPEX = nvdaCF['Change in Fixed Assets & Intangibles']
REVENUE = nvdaIS['Revenue']
NET_INCOME = nvdaCF['Net Income/Starting Line']

In [7]:
data = {'Cash Flow from OA':CF_OA, 'CAPEX': CAPEX, 'FCF to Equity': CF_OA+CAPEX, 'Net Income': NET_INCOME, \
        'Net Income margin': NET_INCOME/REVENUE, 'Revenue':REVENUE}
df = pd.DataFrame(data)

In [8]:
df

,Cash Flow from OA,CAPEX,FCF to Equity,Net Income,Net Income margin,Revenue
in million USD,,,,,,
FY '10,675.797,-97.89,577.907,253.146,0.0714434,3543.31
FY '11,909.156,-138.735,770.421,581.09,0.145348,3997.93
FY '12,824.172,-183.309,640.863,562.536,0.131429,4280.16
FY '13,835,-255,580,440,0.106538,4130
FY '14,906,-122,784,631,0.134771,4682
FY '15,1175,-86,1089,614,0.122555,5010
FY '16,1672,-176,1496,1666,0.2411,6910
FY '17,3502,-593,2909,3047,0.313671,9714
FY '18,3743,-600,3143,4141,0.353448,11716


In [9]:
df['Growth Rate'] = df['Revenue'].pct_change(1)

In [10]:
df

,Cash Flow from OA,CAPEX,FCF to Equity,Net Income,Net Income margin,Revenue,Growth Rate
in million USD,,,,,,,
FY '10,675.797,-97.89,577.907,253.146,0.0714434,3543.31,NaN
FY '11,909.156,-138.735,770.421,581.09,0.145348,3997.93,0.128304
FY '12,824.172,-183.309,640.863,562.536,0.131429,4280.16,0.070594
FY '13,835,-255,580,440,0.106538,4130,-0.035083
FY '14,906,-122,784,631,0.134771,4682,0.133656
FY '15,1175,-86,1089,614,0.122555,5010,0.070056
FY '16,1672,-176,1496,1666,0.2411,6910,0.379242
FY '17,3502,-593,2909,3047,0.313671,9714,0.405789
FY '18,3743,-600,3143,4141,0.353448,11716,0.206094


In [11]:
# FORECASTING for the next 10 years
# FCF/Net Income
coef_FCF_NET_Inco = df['FCF to Equity']/df['Net Income']

for year in range(1,11):
    l = len(df['Growth Rate'])
    if df['Growth Rate'][l-1] < 0 and year ==1:
        min_g = df['Growth Rate'].abs().min()
        rev_est = df['Revenue'][l-1]*(1+min_g)
        net_inc_est = df['Revenue'][l-1]*df['Net Income margin'].mean()
        fcf_est = net_inc_est*coef_FCF_NET_Inco.min()
        row = pd.Series(data = {'FCF to Equity': fcf_est, 'Net Income': net_inc_est, 'Revenue': rev_est, 'Growth Rate': min_g}, name ='Est '+str(year))
        df = df.append(row)                
    else: 
        gr_est = df['Growth Rate'][l-1]*(1 + df['Growth Rate'].mean())
        rev_est = df['Revenue'][l-1]*(1+gr_est)
        net_inc_est = df['Revenue'][l-1]*df['Net Income margin'].mean()
        fcf_est = net_inc_est*coef_FCF_NET_Inco.min()
        row = pd.Series(data = {'FCF to Equity': fcf_est, 'Net Income': net_inc_est, 'Revenue': rev_est, 'Growth Rate': gr_est}, name ='Est '+str(year))
        df = df.append(row)
    

In [12]:
df

,Cash Flow from OA,CAPEX,FCF to Equity,Net Income,Net Income margin,Revenue,Growth Rate
in million USD,,,,,,,
FY '10,675.797,-97.89,577.907,253.146,0.0714434,3543.31,NaN
FY '11,909.156,-138.735,770.421,581.09,0.145348,3997.93,0.128304
FY '12,824.172,-183.309,640.863,562.536,0.131429,4280.16,0.070594
FY '13,835,-255,580,440,0.106538,4130,-0.035083
FY '14,906,-122,784,631,0.134771,4682,0.133656
FY '15,1175,-86,1089,614,0.122555,5010,0.070056
FY '16,1672,-176,1496,1666,0.2411,6910,0.379242
FY '17,3502,-593,2909,3047,0.313671,9714,0.405789
FY '18,3743,-600,3143,4141,0.353448,11716,0.206094


In [39]:
# Required Rate of Return r
r_own = 0.15


# Debt rate rd
interest_expense = 126 # np.abs(nvdaIS['Income Tax (Expense) Benefit, net'][-1])
debt = nvdaBS['Total Liabilities'][-1]
rd = interest_expense/debt

# Tax rate t
income_tax_exp = np.abs(nvdaIS['Income Tax (Expense) Benefit, net'][-1])
income_bef_tax = nvdaIS['Pretax Income (Loss), Adjusted'][-1]
t = income_tax_exp/income_bef_tax

# CAPM for re
Rf = 0.23
Rm = 0.10
re = Rf - beta*(Rm - Rf)

# Weights
Wd = debt/(market_cap+debt)
We = market_cap/(market_cap+debt)

# WACC model r_wacc = Wd*rd*(1-t) + We*re
r_wacc = Wd*rd*(1-t) + We*re

In [68]:
# Terminal Value
FCF_n = df['FCF to Equity'][-1]
V0 = FCF_n*(1 + g_perp)/(r_wacc-g_perp)

# Discount factor
DF = 1+r_wacc

# Fair value of equity
today_value = 0
for year in range(1,11):
    today_value = today_value + df.loc['Est '+str(year)]['FCF to Equity']/DF**year
    
today_value = today_value + V0
fair_value = today_value/shares_out


In [69]:
today_value

11242.98476869555

In [74]:
fair_value


18.281276046659432

In [77]:
def DCF_valuation_method(BS, IS, CF, shares_out, g_perp, market_cap, beta, ret_des, interest_expense, own=True):
    CF_OA = CF['Cash from Operating Activities']
    CAPEX = CF['Change in Fixed Assets & Intangibles']
    REVENUE = IS['Revenue']
    NET_INCOME = CF['Net Income/Starting Line']
    
    # Create DataFrame
    data = {'Cash Flow from OA':CF_OA, 'CAPEX': CAPEX, 'FCF to Equity': CF_OA+CAPEX, 'Net Income': NET_INCOME, \
        'Net Income margin': NET_INCOME/REVENUE, 'Revenue':REVENUE}
    df = pd.DataFrame(data)
    
    # Define Growth rate
    df['Growth Rate'] = df['Revenue'].pct_change(1)
    
    # FORECASTING for the next 10 years
    # FCF/Net Income
    coef_FCF_NET_Inco = df['FCF to Equity']/df['Net Income']

    for year in range(1,11):
         l = len(df['Growth Rate'])
         if df['Growth Rate'][l-1] < 0 and year ==1:
             min_g = df['Growth Rate'].abs().min()
             rev_est = df['Revenue'][l-1]*(1+min_g)
             net_inc_est = df['Revenue'][l-1]*df['Net Income margin'].mean()
             fcf_est = net_inc_est*coef_FCF_NET_Inco.min()
             row = pd.Series(data = {'FCF to Equity': fcf_est, 'Net Income': net_inc_est, 'Revenue': rev_est, 'Growth Rate': min_g}, name ='Est '+str(year))
             df = df.append(row)                
         else: 
             gr_est = df['Growth Rate'][l-1]*(1 + df['Growth Rate'].mean())
             rev_est = df['Revenue'][l-1]*(1+gr_est)
             net_inc_est = df['Revenue'][l-1]*df['Net Income margin'].mean()
             fcf_est = net_inc_est*coef_FCF_NET_Inco.min()
             row = pd.Series(data = {'FCF to Equity': fcf_est, 'Net Income': net_inc_est, 'Revenue': rev_est, 'Growth Rate': gr_est}, name ='Est '+str(year))
             df = df.append(row)
    
    
    # Debt rate rd
    # interest_expense to define # np.abs(nvdaIS['Income Tax (Expense) Benefit, net'][-1])
    debt = BS['Total Liabilities'][-1]
    rd = interest_expense/debt

    # Tax rate t
    income_tax_exp = np.abs(nvdaIS['Income Tax (Expense) Benefit, net'][-1])
    income_bef_tax = nvdaIS['Pretax Income (Loss), Adjusted'][-1]
    t = income_tax_exp/income_bef_tax

    # CAPM for re
    Rf = 0.23
    Rm = 0.10
    re = Rf - beta*(Rm - Rf)

    # Weights
    Wd = debt/(market_cap+debt)
    We = market_cap/(market_cap+debt)

    # WACC model r_wacc = Wd*rd*(1-t) + We*re
    r_wacc = Wd*rd*(1-t) + We*re
    
    
    # Terminal Value
    FCF_n = df['FCF to Equity'][-1]
    V0 = FCF_n*(1 + g_perp)/(r_wacc-g_perp)

    #  Discount factor
    if own:
        DF = 1 + ret_des
    else:
        DF = 1+ r_wacc
    

    # Fair value of equity
    today_value = 0
    for year in range(1,11):
         today_value = today_value + df.loc['Est '+str(year)]['FCF to Equity']/DF**year
    
    today_value = today_value + V0
    fair_value = today_value/shares_out

    return fair_value

In [79]:
fv = DCF_valuation_method(nvdaBS, nvdaIS, nvdaCF, shares_out, g_perp, market_cap,beta, 0.15, 126  )

In [80]:
fv

26.461711615324813